In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
## 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/all you need is attention.pdf")
docs = loader.load()

ImportError: pymupdf package not found, please install it with `pip install pymupdf`

In [5]:
## 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [6]:
## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [7]:
## 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [8]:
## 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [9]:
## 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [10]:
## 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [11]:
## 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "청소년기 자살 관련 뇌부위?"
response = chain.invoke(question)
print(response)

청소년기 자살과 관련된 뇌부위로는 안와전두피질(OFC), 전측 대상피질(ACC), 해마(hippocampus), 복측 전전두피질(ventral PFC), 측두이랑, 선조체(striatum), 배외측 전전두피질(DLPFC), 내측 전전두피질(medial PFC), 후측 대상피질(PCC), 설전부(precuneus), 시상(thalamus) 등이 언급되었습니다. 이러한 뇌부위들은 자살행동과 관련된 정서처리, 인지적 억제능력, 의사결정 과정에서 기능적 이상을 보일 수 있습니다.
